In [15]:
import pandas as pd

In [16]:
df = pd.read_csv(r"C:\Users\spark\Downloads\final_data.csv")


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import random

# Sample Data
selected_columns = ['Title', 'Tag', 'Review', 'Comment', 'Address', 'Country', 'Price', 'Rating', 'tags', 'Governorate']
df = df[selected_columns].dropna()

df['Tag'] = df['Tag'].astype(str)
df['Review'] = df['Review'].astype(str)
df['Comment'] = df['Comment'].astype(str)

# Feature Engineering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Tag'] + ' ' + df['Review'] + ' ' + df['Comment'])

# Function to get recommendations based on user preferences, location, and price
def get_recommendations_with_budget(country, governorate, survey_responses, df, num_days, budget):
    # Filter places that match the user's country and governorate
    filtered_df = df[(df['Country'] == country) & (df['Governorate'] == governorate)]

    # Check if there is data available
    if filtered_df.empty:
        print("No data found for the specified country and governorate.")
        return pd.DataFrame(columns=['Title', 'Price', 'tags', 'Governorate', 'Day'])

    # Ensure "Restaurants" and "Hotels" are always present in user profile
    user_profile = f"{country} {governorate} {' '.join(survey_responses)}"

    # Transform the user profile using the same TF-IDF vectorizer
    user_profile_vectorized = tfidf_vectorizer.transform([user_profile])

    # Transform places' descriptions (assuming 'tags' contains relevant information)
    places_vectorized = tfidf_vectorizer.transform(filtered_df['tags'])

    # Calculate cosine similarity between user profile and places
    sim_scores = linear_kernel(user_profile_vectorized, places_vectorized).flatten()

    # Check if there are places to recommend
    if not any(response_indices for response in survey_responses for response_indices in [i for i, tag in enumerate(filtered_df['tags']) if response.lower() in tag.lower()]):
        print("No suitable places found for the given survey responses.")
        return pd.DataFrame()

    # Calculate the maximum price per day based on the budget and number of days
    max_price_per_day = budget / num_days

    # Initialize recommendations DataFrame
    recommendations_df = pd.DataFrame(columns=['Title', 'Price', 'tags', 'Governorate', 'Day'])

    # Loop through each day
    for day in range(1, num_days + 1):
        # Initialize daily recommendations list
        daily_recommendations = []

        # On the first day, recommend a hotel and two places related to the survey
        if day == 1:
            # Add hotel recommendation for the day
            hotel_recommendation = filtered_df[filtered_df['tags'].str.lower().str.contains('hotel') & (filtered_df['Price'] <= max_price_per_day)].sample(1)[['Title', 'Price', 'tags', 'Governorate']]
            daily_recommendations.append(hotel_recommendation)

        # Add restaurant recommendation for each day
        restaurant_recommendation = filtered_df[filtered_df['tags'].str.lower().str.contains('restaurant') & (filtered_df['Price'] <= max_price_per_day)].sample(1)[['Title', 'Price', 'tags', 'Governorate']]
        daily_recommendations.append(restaurant_recommendation)

        # Add two places related to the survey for each day
        for response in survey_responses:
            # Get indices of places containing the current survey response
            response_indices = [i for i, tag in enumerate(filtered_df['tags']) if response.lower() in tag.lower()]

            # If there are places with the current tag, select a random one within the budget
            if response_indices:
                valid_indices = [idx for idx in response_indices if filtered_df.iloc[idx]['Price'] <= max_price_per_day]
                if valid_indices:
                    random_index = random.choice(valid_indices)
                    recommendation = filtered_df.iloc[[random_index]][['Title', 'Price', 'tags', 'Governorate']]
                    daily_recommendations.append(recommendation)

        # Add the daily recommendations to the overall recommendations DataFrame
        for recommendation in daily_recommendations:
            recommendation['Day'] = day
            recommendations_df = pd.concat([recommendations_df, recommendation])

        # Print recommendations for the day
        print(f"\nRecommendations for Day {day}:")
        for i, recommendation in enumerate(daily_recommendations):
            print(f"\nRecommendation {i + 1}:")
            print(f"Title: {recommendation['Title'].values[0]}")
            print(f"Price: {recommendation['Price'].values[0]}")
            print(f"Tags: {recommendation['tags'].values[0]}")
            print(f"Governorate: {recommendation['Governorate'].values[0]}")

    return recommendations_df

# Example: Get recommendations for a user with a specific country, governorate, survey responses, number of days, and budget
user_country = "Egypt"
user_governorate = "Alexandria"
user_survey_responses = ["Museums", "water places", "for fun","Archaeological tourism","Religious Tourism","malls ","parks","Tours","Natural views"]
num_days = int(input("Enter the number of days you want to spend: "))
budget = float(input("Enter your budget: "))

recommendations = get_recommendations_with_budget(user_country, user_governorate, user_survey_responses, df, num_days, budget)



Enter the number of days you want to spend: 2
Enter your budget: 300

Recommendations for Day 1:

Recommendation 1:
Title: Africa Safari Park & Motel
Price: 34.77
Tags: Hotel
Governorate: Alexandria

Recommendation 2:
Title: Nonna Cucina
Price: 32.21
Tags: Restaurant
Governorate: Alexandria

Recommendation 3:
Title: Alexandria National Museum
Price: 71.13
Tags: Museums
Governorate: Alexandria

Recommendation 4:
Title: Fort Qaitbey
Price: 31.8
Tags: Archaeological tourism
Governorate: Alexandria

Recommendation 5:
Title: Saint Mark's Coptic Orthodox Cathedral
Price: 36.13
Tags: Religious Tourism
Governorate: Alexandria

Recommendation 6:
Title: Travelish Tours
Price: 68.15
Tags: Tours
Governorate: Alexandria

Recommendations for Day 2:

Recommendation 1:
Title: Cap D'or
Price: 36.26
Tags: Restaurant
Governorate: Alexandria

Recommendation 2:
Title: Royal Jewelry Museum
Price: 61.67
Tags: Museums
Governorate: Alexandria

Recommendation 3:
Title: Catacombs of Kom el Shoqafa
Price: 131.4
T